# Anna Hajduk - Homework 1
### Possibilistic k-means implementation


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

X=np.array([(4,1940),(9,2960),(9,4630),(78,1528),(90,2040),(50,3700),(467,14815),(509,15200),(290,15700),(215,6045)])
x1 = np.array(X[:,0])
x2 = np.array(X[:,1])


fig, ax = plt.subplots()
ax.scatter(x1,x2)
ax.set(xlabel='Seats count', ylabel='Distance range (km)',
       title='Aircrafts')
ax.grid()
plt.show()

<Figure size 640x480 with 1 Axes>

In [2]:
#normalization
train_data = np.array(X)
max_values = train_data.max(0)

data_set = np.divide(train_data,max_values)

In [3]:
m = 2
error_margin = 0.01

#random assignation
centers = []
groups = 2
assignation = np.random.rand( len(data_set), groups)

print(assignation)

[[0.13991647 0.28402215]
 [0.62577796 0.35558305]
 [0.11525589 0.19774058]
 [0.10515582 0.40145995]
 [0.73430283 0.61342314]
 [0.54757902 0.33015723]
 [0.72219862 0.08568255]
 [0.34693739 0.21531211]
 [0.93002955 0.21399974]
 [0.66527404 0.77296671]]


### Mahalanobis distance implementation

In [4]:
def mahalanobis_distance(x, v):
    sub = x - v
    inv = np.identity(len(x))
    return np.sqrt(np.dot(np.dot(sub.T, inv), sub))

### Calculate_eta implementation

In [5]:
def D2ikA(i, k):
    return mahalanobis_distance(data_set[k], centers[i])

In [6]:
def calculate_eta(i):
    numerator = 0
    denominator = 0
    for k in range(assignation.shape[0]):
        numerator += assignation[k, i]**m * D2ikA(i, k)**2
        denominator += assignation[k, i]**m
    
    assert(denominator != 0)
    return numerator / denominator

### Calculate_u implementation

In [7]:
def uik(i, k):
    eta = calculate_eta(i)
    return (1 + (D2ikA(i, k)/ calculate_eta(i))**(2/(m-1) )) ** -1

In [8]:
def calculate_u():
    new_asignations = np.zeros(assignation.shape)
    for i in range(groups):
        for k in range(new_asignations.shape[0]):
            new_asignations[k, i] = uik(i, k)
    return new_asignations

### Centers

In [9]:
centers = []
space=[[0,1],[0,1]]

def select_centers():
    global centers
    global groups
    global space
    iter=0
    while iter<groups:
        centers.append((random.uniform(space[0][0],space[0][1]), 
                        random.uniform(space[1][0],space[1][1])))
        iter += 1

In [10]:
select_centers()
print(centers)

[(0.3200827596615802, 0.15682728932843015), (0.32940278498844444, 0.43824383188972926)]


In [11]:
def calculate_new_centers(u):
    global centers
    new_centers=[]
    for k in range(groups):
        u_x_vector=np.zeros(2)
        u_scalar=0.0
        for i in range(len(data_set)):
            u_scalar = u_scalar+(u[i][k]**m)
            u_x_vector=np.add(u_x_vector,np.multiply(u[i][k]**m,data_set[i]))
        assert(u_scalar != 0)
        new_centers.append(np.divide(u_x_vector,u_scalar))
    centers=new_centers

In [12]:
def calculate_differences(new_assignation):
    global assignation    
    return np.sum(np.abs(np.subtract(assignation,new_assignation)))

In [ ]:
def cluster():
    global assignation
    global error_margin    
    global groups
    
    difference_limit_not_achieved=True
    iter=0
    while difference_limit_not_achieved:
 
        new_assignation = calculate_u()
        calculate_new_centers(new_assignation)

        if iter>0:
            if calculate_differences(new_assignation) < error_margin:
                difference_limit_not_achieved=False
        assignation=new_assignation
        iter=iter+1
        
cluster()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
print(centers)

In [ ]:
print(assignation)

In [ ]:
red = []
blue = []

i = 0
for p1, p2 in assignation:
    if p1 > p2:
        red.append(data_set[i])
    else:
        blue.append(data_set[i])
    i += 1
    
red = np.array(red)
blue = np.array(blue)

In [ ]:
fig, ax = plt.subplots()

ax.scatter(blue[:,0],blue[:,1],c='blue')
ax.scatter(red[:,0],red[:,1],c='red')
ax.scatter(np.array(centers)[:,0],np.array(centers)[:,1],c='black')
ax.set(xlabel='Seats count', ylabel='Distance range (km)',
       title='Aircrafts (clusters)')
ax.grid()
plt.show()

In [ ]:
data_set

In [ ]:
assignation.shape